In [4]:
import pandas as pd
import gspread
import os
from IPython.display import display
from datetime import datetime, date, time

# (Presume que 'google-auth' etc. estão instalados)

def ler_planilha_google(sheet_key, worksheet_name, credentials_path='credentials.json'):
    """
    Lê os dados de uma aba específica de uma Planilha Google.
    Assume cabeçalhos na Linha 1 e dados na Linha 2.
    """
    print(f"\n--- Iniciando leitura da Planilha Google ---")
    print(f"  Planilha Key: {sheet_key}")
    print(f"  Aba         : {worksheet_name}")

    if not os.path.exists(credentials_path):
        print(f"ERRO: Arquivo de credenciais '{credentials_path}' não encontrado.")
        return None

    try:
        # 1. Autenticar
        print("Autenticando com conta de serviço...")
        gc = gspread.service_account(filename=credentials_path)
        print("Autenticação OK.")

        # 2. Abrir a Planilha
        print(f"Abrindo planilha...")
        sh = gc.open_by_key(sheet_key)
        print("Planilha aberta.")

        # 3. Selecionar a Aba
        try:
            worksheet = sh.worksheet(worksheet_name)
            print(f"Aba '{worksheet_name}' encontrada.")
        except gspread.exceptions.WorksheetNotFound:
            print(f"ERRO: Aba '{worksheet_name}' não encontrada na planilha.")
            return None

        # 4. Ler Todos os Valores
        print(f"Lendo todos os dados da aba '{worksheet_name}'...")
        all_values = worksheet.get_all_values(value_render_option='FORMATTED_VALUE')
        
        if not all_values or len(all_values) < 2:
             print("Aviso: Aba está vazia ou contém apenas cabeçalho.")
             return pd.DataFrame()
             
        headers = all_values[0] # Linha 1 (índice 0) como cabeçalho
        data = all_values[1:]   # Dados da Linha 2 (índice 1) em diante

        # Cria o DataFrame
        print("Criando DataFrame Pandas...")
        df = pd.DataFrame(data, columns=headers)
        print("DataFrame criado com sucesso.")
        return df

    except Exception as e:
        print(f"ERRO inesperado durante a leitura da Google Sheet: {e}")
        return None

In [ ]:
import pandas as pd
import os
# (Presume que 'ler_planilha_google' está definida em outra célula)

def buscar_sincronizacoes_automaticas(sheet_key, worksheet_name, credentials_path, 
                                            coluna_filtro='Quem', 
                                            valor_filtro="**SINCRONIZAÇÃO EM...**"):
    """
    (VERSÃO MODIFICADA)
    Lê a aba especificada do Google Sheet e retorna apenas as linhas
    que COMEÇAM COM o 'valor_filtro' (comparando os 15 primeiros caracteres)
    na 'coluna_filtro'.

    Args:
        sheet_key (str): Chave da Planilha Google.
        worksheet_name (str): Nome da aba para ler.
        credentials_path (str): Caminho para o arquivo de credenciais.
        coluna_filtro (str): Nome da coluna para aplicar o filtro (padrão: 'Quem').
        valor_filtro (str): O *início* do texto para filtrar (padrão: '**SINCRONIZAÇÃO AUTOMÁTICA**').
                            A comparação usará os 15 primeiros caracteres.

    Returns:
        pd.DataFrame: Um DataFrame contendo apenas as linhas filtradas.
                      Retorna um DataFrame vazio se nada for encontrado ou em caso de erro.
    """
    
    # Mensagem de print atualizada para refletir a nova lógica
    print(f"Buscando linhas onde '{coluna_filtro}' começa com '{valor_filtro[:15]}...' na aba '{worksheet_name}'...")
    
    # 1. Ler a planilha inteira (presume que 'ler_planilha_google' existe)
    df_completo = ler_planilha_google(sheet_key, worksheet_name, credentials_path)
    
    # 2. Validar o DataFrame
    if df_completo is None or df_completo.empty:
        print("Não foi possível ler a planilha ou ela está vazia. Retornando DataFrame vazio.")
        return pd.DataFrame() # Retorna DF vazio

    # 3. Validar a coluna de filtro
    if coluna_filtro not in df_completo.columns:
        print(f"ERRO: A coluna de filtro '{coluna_filtro}' não foi encontrada na planilha.")
        return pd.DataFrame() # Retorna DF vazio

    # --- 4. APLICAR O FILTRO (LÓGICA MODIFICADA) ---
    try:
        # Pega os 15 primeiros caracteres do valor_filtro, em minúsculas
        valor_filtro_lower_15 = valor_filtro.lower()[:15]
        
        # Converte a coluna para string (lidando com NaNs),
        # coloca em minúsculas, e pega os 15 primeiros caracteres
        coluna_para_comparar = df_completo[coluna_filtro].astype(str).str.lower().str[:15]
        
        # Compara a fatia da coluna com a fatia do valor_filtro
        df_filtrado = df_completo[
            coluna_para_comparar == valor_filtro_lower_15
        ].copy() # .copy() para evitar warnings
        
        if df_filtrado.empty:
            print("Nenhuma linha encontrada correspondente ao filtro.")
        else:
            print(f"Encontradas {len(df_filtrado)} linhas correspondentes.")
            
        return df_filtrado
        
    except Exception as e:
        print(f"ERRO ao aplicar o filtro: {e}")
        return pd.DataFrame() # Retorna DF vazio

In [10]:
# --- Configuração (Conforme fornecido por você) ---
SHEET_KEY = '1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY'
CREDS_FILE = 'credentials2.json'
WORKSHEET_NAME = 'Reservas Consolidadas'

# --- Execução ---
# (Assumindo que o valor 'SINCRONIZACAO AUTOMATICA' está na coluna 'Origem')
df_sincronizacoes = buscar_sincronizacoes_automaticas(
    SHEET_KEY, 
    WORKSHEET_NAME, 
    CREDS_FILE
)

# --- Verificação ---
if df_sincronizacoes is not None and not df_sincronizacoes.empty:
    print(f"\n--- Exibindo {len(df_sincronizacoes)} Linhas de Sincronização Automática ---")
    display(df_sincronizacoes)
else:
    print("\nNenhum dado de sincronização automática foi encontrado.")

Buscando linhas onde 'Quem' é '**SINCRONIZAÇÃO AUTOMÁTICA**' na aba 'Reservas Consolidadas'...

--- Iniciando leitura da Planilha Google ---
  Planilha Key: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY
  Aba         : Reservas Consolidadas
Autenticando com conta de serviço...
Autenticação OK.
Abrindo planilha...
Planilha aberta.
Aba 'Reservas Consolidadas' encontrada.
Lendo todos os dados da aba 'Reservas Consolidadas'...
Criando DataFrame Pandas...
DataFrame criado com sucesso.
Encontradas 2 linhas correspondentes.

--- Exibindo 2 Linhas de Sincronização Automática ---


,idReserva,Apartamento,Início,Fim,Dias,Pessoas,Quem,Origem,Recebido,Despesas,A receber,Total Lq,Total BT,Diária BT,Diária Lq,Com.,Diária BT PAX,Diária Lq PAX,Status
1543,1601,AP-201,25/02/2026 16:00,26/02/2026 11:00,,,**SINCRONIZAÇÃO AUTOMÁTICA**,Direto,,,,,,,,,,,
1551,466,CBL004,24/03/2026 16:00,27/03/2026 11:00,,,**SINCRONIZAÇÃO AUTOMÁTICA**,Airbnb,,,,,,,,,,,
